<a href="https://colab.research.google.com/github/georgeliu1998/sentiment_word_embedding/blob/master/sentiment_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy.random import seed
seed(8)

from tensorflow import set_random_seed
set_random_seed(8)

import numpy as np
seed = np.random.RandomState(8)

import urllib.request
import tarfile
import os
import re
import string
from time import time

import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
!python -m spacy download en_core_web_md

    100% |████████████████████████████████| 120.9MB 52.3MB/s 
  Running setup.py install for en-core-web-md ... - \ | / - \ | / - \ done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

path = './gdrive/My Drive/WorkingDir/sentiment_word_embedding'
download_folder = 'raw_data'
download_name = 'imdb.tar.gz'
path_name = os.path.join(path, download_folder, download_name)

# Download data to the specified path
urllib.request.urlretrieve(url, path_name)

('./gdrive/My Drive/WorkingDir/sentiment_word_embedding/raw_data/imdb.tar.gz',
 <http.client.HTTPMessage at 0x7fa4dbb55898>)

In [0]:
#extract_path = os.path.join(path, download_folder)

with tarfile.open(path_name) as tar:
  #tar.extractall(path=extract_path)
  tar.extractall()

In [0]:
def load_imdb(path):
  """
  Loads train and test data into dataframes.

  Params
  ---------------------
  path: str
    The path to the unzipped aclImdb folder.

  Returns
  ---------------------
  df_train, df_test: tuple of of pandas df
    The dataframes created from data
  """

  data = {}

  for split in ['train', 'test']:
    data[split] = []

    for label in ['pos', 'neg']:
      sentiment = 1 if label == 'pos' else 0
      file_names = os.listdir(os.path.join(path, split, label))

      for file_name in file_names:
        file_path = os.path.join(path, split, label, file_name)
        with open(file_path, "r") as f:
          review = f.read()

          data[split].append([review, sentiment])

  np.random.shuffle(data['train'])        
  df_train = pd.DataFrame(data['train'], columns=['review', 'sentiment'])

  np.random.shuffle(data['test'])
  df_test = pd.DataFrame(data['test'], columns=['review', 'sentiment'])

  return df_train, df_test

In [0]:
#data_path = os.path.join(extract_path, 'aclImdb')
df_train, df_test = load_imdb('./aclImdb')

In [0]:
def save_df(df, path, save_name):
  file_name = save_name + '.csv'
  path_name = os.path.join(path, file_name)
  
  df.to_csv(path_name, index=False)

In [0]:
save_df(df=df_train, path=path, save_name='df_train')
save_df(df=df_test, path=path, save_name='df_test')

In [0]:
path = './gdrive/My Drive/WorkingDir/sentiment_word_embedding'

df_train = pd.read_csv(os.path.join(path, 'df_train.csv'))
df_test = pd.read_csv(os.path.join(path, 'df_test.csv'))

In [36]:
df_train[df_train.review.str.contains('<.*?>')]['review'].iloc[60]

'The movie appeals to public due to charisma of Ben Stiller and notoriety of J. Aniston. It seems that we have here a recipe for a successful title, but there\'s nothing successful in this movie.<br /><br />Polly is very well played by Aniston, no doubt. This is the kind of character which suits her perfectly. <br /><br />Bem Stiller is the same troublesome guy like in " Meet the parents", but in this movie the comic scenes are few compared to the title mentioned above.<br /><br />The script is very poor with nothing special at all. With this two well payed actors the things could get a lot better - but what can they do when there is such a poor story and script.<br /><br />4 out of 10.'

In [0]:
def measure_time(func):
  """
  Tests how long a given function takes to run.
  
  """
  t1 = time()
 
  func
  
  t2 = time()
  
  t_sec = t2 - t1 
  t_min = t_sec / 60

  print("It took {:.2f} seconds or {:.2f} minutes.".format(t_sec, t_min))

In [0]:
def preprocess_text(text, lemmatizing=False):
    """
    Preprocesses text by removing all stop words and lemmatizing.
    
    Params
    --------------
    text: str
      the string to be cleaned
    
    Returns
    --------------
    text: str
      the cleaned string

    """
    text = text.lower()
    
    if lemmatizing:
      lemmatizer = WordNetLemmatizer()
      text = ' '.join([lemmatizer.lemmatize(word, pos='v') for word in text.split()])
    
    # Remove html tags
    text = re.sub(r'<.*?>', '', text)
    
    # Replace punctuation with spaces
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(translator)

    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    
    # Remove additional white spaces
    text = ' '.join(text.split())
    
    return text

In [71]:
t1 = time()

X_train = df_train['review'].apply(preprocess_text)

t2 = time()

t = (t2 - t1) / 60

print('Took {} min'.format(t))

Took 14.381852738062541 min


In [73]:
df_train['review_processed'] = X_train
df_train.head()

,review,sentiment,review_processed
0,I could never remember the name of this show. ...,1,could never remember name show use watch 8 rem...
1,"Going into this movie, I had heard good things...",1,going movie heard good things coming really am...
2,The only scary thing about this movie is the t...,0,scary thing movie thought whoever made might m...
3,This deserves a 12 out of 10. An absolutely re...,1,deserves 12 10 absolutely refreshing show real...
4,"Joan Fontaine is ""A Damsel in Distress"" in thi...",1,joan fontaine damsel distress 1937 musical sta...


In [0]:
save_df(df=df_train, path=path, save_name='df_train')

In [78]:
t1 = time()

df_test['review_processed'] = df_test['review'].apply(preprocess_text)

t2 = time()

t = (t2 - t1) / 60

save_df(df=df_test, path=path, save_name='df_test')

print('Took {} min'.format(t))

Took 13.990289672215779 min


In [79]:
df_test.head()

,review,sentiment,review_processed
0,"""Thunderbolt"" is probably Jackie Chan's worst ...",0,thunderbolt probably jackie chan worst movie s...
1,This is the worst movie that I have ever seen....,0,worst movie ever seen first thought going good...
2,(Warning: Some spoilers ahead.)<br /><br />Wha...,0,warning spoilers ahead incredibly crappy movie...
3,This film is a perfect example of how to take ...,0,film perfect example take fascinating subject ...
4,"Ok, everybody agreed on what was the best seas...",1,ok everybody agreed best season first killing ...


In [83]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer()

training_features = vectorizer.fit_transform(df_train['review_processed'])    
test_features = vectorizer.transform(df_test['review_processed'])

# Training
model = LinearSVC()
model.fit(training_features, df_train['sentiment'])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test['sentiment'], y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

Accuracy on test set: 84.14%


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

In [0]:
X_train = np.array(list(train['text'].str.lower().apply(lambda x: nlp(x).vector.tolist())))
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_train.csv", X_train, delimiter=",")

In [0]:
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_train.csv", y_train, delimiter=",")

In [0]:
X_train = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_train.csv", delimiter=",")
y_train = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_train.csv", delimiter=",")
X_test = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_test.csv", delimiter=",")
y_test = np.loadtxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_test.csv", delimiter=",")

In [45]:
# Get the first 1000 samples from train dataset
#X_train, y_train = X_train[:1000,], y_train[:1000]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 300), (25000,), (25000, 300), (25000,))

In [49]:
model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=100,
          batch_size=128)

score = model.evaluate(X_test, y_test, batch_size=128)
print("Accuracy on the test set is: {}".format(score[1]))

Epoch 1/100
25000/25000 [==============================] - 1s 42us/step - loss: 0.5857 - acc: 0.7167
Epoch 2/100
25000/25000 [==============================] - 1s 29us/step - loss: 0.4721 - acc: 0.7886
Epoch 3/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.4276 - acc: 0.8115
Epoch 4/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.4074 - acc: 0.8229
Epoch 5/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3937 - acc: 0.8285
Epoch 6/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3855 - acc: 0.8342
Epoch 7/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3802 - acc: 0.8358
Epoch 8/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3749 - acc: 0.8395
Epoch 9/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3720 - acc: 0.8401
Epoch 10/100
25000/25000 [==============================] - 1s 33us/step - loss: 0.3675 - a

In [0]:
def convert_to_embedding(col):
  """
  Converts a pandas column into a numpy array by concatenating average embedding
  vectors for all rows in the column
  """

  return np.array(list(col.str.lower().apply(lambda x: nlp(x).vector.tolist())))

In [0]:
col = test['text']

t1 = time()
X_test = convert_to_embedding(col)
t2 = time()

t = t2 - t1

t /= 60

print("Finished cross-valiation. Took {:.1f} min.".format(t))

Finished cross-valiation. Took 37.5 min.


In [0]:
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/X_test.csv", X_test, delimiter=",")

In [0]:
y_test = test['sentiment'].values
np.savetxt("./gdrive/My Drive/WorkingDir/sentiment_word_embedding/y_test.csv", y_test, delimiter=",")

In [41]:
#X_test, y_test = X_test[:1000,], y_test[:1000]
X_test.shape, y_test.shape

((1000, 300), (1000,))

## To-Do's
 
- preprocess using spaCy
- lemmatize based on pos: https://stackoverflow.com/questions/41824782/lemmatize-string-according-to-pos-nlp

In [0]:
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [0]:

from keras.layers import Input,Dense
from keras.models import Model
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
 
(data,labels),(x_test,y_test) = mnist.load_data()
 
x_train = data.reshape(len(data),-1)
y_train = np_utils.to_categorical(labels, 10)
 
# This returns a tensor
inputs = Input(shape=(784,))
 
# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
 
# This creates a model that includes the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train)  # starts training

11493376/11490434 [==============================] - 1s 0us/step
Epoch 1/1
60000/60000 [==============================] - 19s 313us/step - loss: 6.1711 - acc: 0.6105


In [0]:
data.shape, labels.shape

((60000, 28, 28), (60000,))

In [0]:
x_train.shape, y_train.shape

((60000, 784), (60000, 10))